## Retrieve with RA

### import

In [3]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
from utils.search_address import RetrieveAddress, get_lat_long

In [4]:
df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
df_addr = pd.read_csv('./address/address.csv')

### Problem in Retrieving

주소 데이터의 최소 유닛은 '동' 혹은 '리' 이다.  

부산, 김제시 같은 경우에는 최소 유닛이 아니므로,  
최소 유닛 중에 첫번째 열을 결과로 가져온다.  

어떻게 해야할까?  

#### level 2 problem

#### level 1 problem

In [5]:
history = []
for item in df_where[:1]:
    if isinstance(item, str):
        ic(item)
        ra = RetrieveAddress(df_addr)
        res = ra.process(item)
        history.append(ra)
        ic(res)

ic| item: '부산의 한 모텔'
ic| res: ''


#### making a set of lv.1

In [6]:
df_addr['lv1'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 28199 entries, 0 to 28198
Series name: lv1
Non-Null Count  Dtype 
--------------  ----- 
28199 non-null  object
dtypes: object(1)
memory usage: 220.4+ KB


In [9]:
df_addr['lv1'].index

RangeIndex(start=0, stop=28199, step=1)

In [12]:
lv1_set = set([])

# from tqdm import tqdm

# for index in tqdm(df_addr.index):
for index in df_addr.index:
    value = df_addr.iloc[index]['lv1']
    if value not in lv1_set:
        lv1_set.add(value)

In [ ]:
lv1_set_additional = [
    '강원', '경기', '경남', '경북', '광주',
    '대구', '대전', '부산', '서울', '울산',
    '인천', '전남', '전북', '제주', '충남',
    '충북'
]

lv1_set.update(lv1_set_additional)

In [17]:
lv1_set

{'강원',
 '강원특별자치도',
 '경기',
 '경기도',
 '경남',
 '경북',
 '경상남도',
 '경상북도',
 '광주',
 '광주광역시',
 '대구',
 '대구광역시',
 '대전',
 '대전광역시',
 '부산',
 '부산광역시',
 '서울',
 '서울특별시',
 '울산',
 '울산광역시',
 '인천',
 '인천광역시',
 '전남',
 '전라남도',
 '전북',
 '전북특별자치도',
 '제주',
 '제주특별자치도',
 '충남',
 '충북',
 '충청남도',
 '충청북도'}

#### level 1 problem - try 1

In [3]:
history = []
for item in df_where[:1]:
    if isinstance(item, str):
        ic(item)
        ra = RetrieveAddress(df_addr)
        res = ra.process(item)
        history.append(ra)
        ic(res)

ic| item: '부산의 한 모텔'
ic| res: '울산광역시'


#### fix - search_in_lv1()

In [4]:
ra = RetrieveAddress(df_addr)

In [5]:
ra.set_text('부산의 한 모텔')

In [6]:
def search_in_lv1():
    for item in ra.lv1_set:
        if re.search(
            r'\b{}'.format(item),
            ra.text,
            re.IGNORECASE):
            ic(item)

In [7]:
search_in_lv1()

ic| item: '부산'


#### level 1 problem - try 2

In [3]:
history = []
for item in df_where[:1]:
    if isinstance(item, str):
        ic(item)
        ra = RetrieveAddress(df_addr)
        res = ra.process(item)
        history.append(ra)
        ic(res)

ic| item: '부산의 한 모텔'
ic| res: '부산광역시'


희한하네?  
'부산'이 아니고, '부산광역시' ??  
왜지?  

#### level 1 problem - try 3

In [3]:
history = []
for item in df_where[:1]:
    if isinstance(item, str):
        ic(item)
        ra = RetrieveAddress(df_addr)
        res = ra.process(item)
        history.append(ra)
        ic(res)

ic| item: '부산의 한 모텔'
ic| res: '부산'


`r'\b{}'.format('|'.join(item)),`  
이렇게 되어 있었네.

`r'\b{}'.format(item),`  
바꾸니까 정상.

### retrieve test

#### load dataframe - 'df'

In [4]:
df_news = load_var("df")

In [5]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   범죄 유형             291 non-null    object 
 1   지역                107 non-null    object 
 2   위도                35 non-null     float64
 3   경도                35 non-null     float64
 4   중복 여부             191 non-null    object 
 5   퀄리티               47 non-null     object 
 6   제외 여부             291 non-null    object 
 7   기사제목              291 non-null    object 
 8   사건 장소             289 non-null    object 
 9   수사 기관             290 non-null    object 
 10  본문                291 non-null    object 
 11  URL               290 non-null    object 
 12  일자                291 non-null    int64  
 13  언론사               291 non-null    object 
 14  기고자               283 non-null    object 
 15  비고                68 non-null     object 
 16  content           290 non-null    object 
 1

In [9]:
df_news.iloc[1]['content_text']

'\n\n\n술 취한 유부남 간부, 길거리서 여성에 \'강제 입맞춤\'…대만 발칵 - 아시아경제\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\nbar_progress\n\n\n\n\n\n\n\n\n아시아경제\n\n\n\n\n\n\n사이트맵메뉴 열기\n\n\n\n\n\n증권\n\n\n\n\n경제\n\n\n\n\n부동산\n\n\n\n\n산업·IT\n\n\n\n\n정치\n\n\n\n\n사회\n\n\n\n\n국제\n\n\n\n\n문화·라이프\n\n\n\n\n오피니언\n\n\n\n\n\n\n법률소식\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\t실시간 뉴스\n\t\t\t\t\t\t\n\n\n\n\n\n김정호 민주당 김해을 후보 “검증된 뚝심·실력으로 김해 발전 이끌 것”\n\n\n\n\n수익률 7％·수수료 0원 … 근로복지공단 창원지사 “푸른씨앗, 퇴직연금 관리에 딱!”\n\n\n\n\n합천 공장서 ‘불’ … 야산으로 번져 1시간여 만에 진화\n\n\n\n\n김병규 진주을 무소속 후보 “우주항공청 업무 총괄한 진짜 일꾼”\n\n\n\n\n박완수 경남지사, 부산항 신항 ‘서컨’ 점검 “항만산업·항만 인재 선제 육성해야”\n\n\n\n\n경기 총선 후보 평균 재산 50억원↑…김복덕·안철수 등\n\n\n\n\n"애들이 마시면 어쩌려고" 수입산 커피서 발기부전 치료제 검출\n\n\n\n\n종로 7:1 최고…총선 평균 경쟁률은 2.7대 1\n\n\n\n\n中 서열 4·5위, 北김성남 만나…"평화·안정 외부환경 조성"\n\n\n\n\n英 중앙은행 총재, 올해 금리인하 기대 "비합리적이지 않아"\n\n\n\n\n\n\n\n\n국제\n\n술 취한 유부남 간부, 길거리서 여성에 \'강제 입맞춤\'…대만

#### inspection dataframe - df_hk_json

In [10]:
df_hk_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   범죄 유형   0 non-null      float64
 1   지역      3 non-null      object 
 2   위도      0 non-null      float64
 3   경도      0 non-null      float64
 4   중복 여부   6 non-null      object 
 5   퀄리티     0 non-null      float64
 6   제외 여부   0 non-null      float64
 7   기사제목    27 non-null     object 
 8   사건 장소   27 non-null     object 
 9   수사 기관   27 non-null     object 
 10  본문      27 non-null     object 
 11  URL     27 non-null     object 
 12  일자      27 non-null     int64  
 13  언론사     27 non-null     object 
 14  기고자     27 non-null     object 
 15  비고      0 non-null      float64
 16  text    27 non-null     object 
 17  who     5 non-null      object 
 18  when    5 non-null      object 
 19  where   5 non-null      object 
 20  what    5 non-null      object 
 21  json    27 non-null     object 
dtypes: f

In [26]:
df_hk_json[df_hk_json['지역'].isna() == False]

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,...,일자,언론사,기고자,비고,text,who,when,where,what,json
1,NaN,전라북도 김제시 금산면,NaN,NaN,NaN,NaN,NaN,"""여자 혼자 있는 것 같아서"" 무단 침입해 주먹 휘두른 60대男",전북,"김제경찰서,경찰,김제시,한경닷컴 객원",...,20230413,한국경제,이보배,NaN,/사진=게티이미지뱅크\n일면식도 없는 여성의 집에 무단 침입해 주먹을 휘두른 60대...,60대 남성,오후 2시,김제시의 한 아파트,주먹을 휘두른 60대 여성의 집에 무단 침입해 주먹을 휘두른 혐의(주거침입 등),"{""who"": ""60대 남성"", ""when"": ""오후 2시"", ""where"": ""김..."
2,NaN,서울시 관악구,NaN,NaN,NaN,NaN,NaN,길거리서 웃통 벗고 여성에 달려든 문신男 경찰 오자 '난동',"서울,관악구","유튜브,서울경찰청",...,20230519,한국경제,김소연,NaN,/사진=유튜브 채널 '서울경찰' 영상 캡처\n출근길 '묻지마 폭행'을 저지른 남성이...,남성,7일 오전 2시 45분,서울 강남구 압구정동의 길가,헌팅을 거절한 여성에 격분한 남성이 폭행을 저지르는 모습,"{""who"": ""남성"", ""when"": ""3월 서울 관악구 골목길에서 난동을 부렸던..."
5,NaN,서울시 강남구 압구정동,NaN,NaN,NaN,NaN,NaN,"""같이 한잔해"" 끝없는 대쉬 평일 밤에도 '헌팅포차' 꽉 찼다 [여기잇슈]","라운지,펀치남,서울,압구정,강남역,강남,인근","압구정로데오,정부,한잔해,SBS,BJ,한창",...,20230524,한국경제,김세린,NaN,강남역의 '헌팅 포차' 거리 한 술집에 야외 테이블까지 사람들이 가득 차 있는 모습...,NaN,NaN,NaN,NaN,"{\n""who"": ""한 남성 2명과 여성 2명"",\n""when"": ""근일 밤"",\n..."


#### text from df_hk_json

In [7]:
texts = df_hk_json.iloc[:]['text']

In [28]:
texts[2]

'/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n출근길 \'묻지마 폭행\'을 저지른 남성이 현행범으로 체포되는 모습이 공개됐다.\n\n18일 서울경찰청 공식 유튜브 채널에 \'출근길 묻지마 폭행범 검거 현장\'이라는 제목으로 지난 3월 서울 관악구 골목길에서 난동을 부렸던 남성의 모습이 담긴 영상이 게재됐다.\n\n이 남성은 상의를 벗어 문신이 그려진 몸으로 길을 걷던 한 여성을 향해 달려갔다. 갑자기 돌진하는 남성을 보며 놀란 여성이 횡단보도로 뛰어가며 도망쳤지만, 남성은 그 뒤를 쫓아갔다.\n\n다른 행인들이 남성을 말렸음에도 난동은 이어졌다. 신고받고 현장에 출동한 경찰은 폭행 및 공무집행방해 등 혐의로 현장에서 체포했다.\n/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n경찰의 등장에도 남성은 침을 뱉고, 욕을 하는 등 폭력적인 행동을 이어갔다. 결국 경찰은 남성을 힘으로 제압해 압송했다.\n\n경찰에 따르면 이 남성과 여성은 일면식도 없는 사이였다. 경찰은 "마약사범으로 의심했으나 마약 성분은 검출되지 않았다"며 "관제센터와의 공조, 시민들의 도움으로 빠르게 찾아낸 덕에 2차 피해를 막을 수 있었다"고 전했다.\n\n김소연 한경닷컴 기자 sue123@hankyung.com'

#### RA processing

In [22]:
history = []
for item in tqdm(texts):
    if isinstance(item, str):
        # ic(item)
        ra = RetrieveAddress(df_addr)
        res = ra.process(item)
        history.append(ra)
        # ic(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [03:05<00:00,  6.88s/it]


기사 하나당 7초쯤 걸린다.  
RAG 안 쓰고 이렇게 하는 게 더 낫겠다.  
적어도 주소를 뽑아낼 때는.  

#### save 'ra_history'

In [25]:
save_var(history, "ra_history")

In [6]:
ra_history = load_var("ra_history")

#### save address in 'ra_address' column

In [29]:
for idx, item in enumerate(ra_history):
    # ic(item.address)
    df_hk_json.loc[idx, 'ra_address'] = item.address

/var/folders/r0/w084nh2s0v3fstg3n70027500000gn/T/ipykernel_57912/3298260219.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '부산' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_hk_json.loc[idx, 'ra_address'] = item.address


In [44]:
save_var(df_hk_json, 'df_hk_json')

In [5]:
df_hk_json = load_var('df_hk_json')

#### inspection 'ra_address' column with 'text' column

In [6]:
index = 2
columns = ['text', 'ra_address']
ic(df_hk_json[columns].loc[index, 'ra_address'])
print(df_hk_json[columns].loc[index, 'text'].replace('\n\n', '\n'))

ic| df_hk_json[columns].loc[index, 'ra_address']: '전라남도 순천시 행동'


/사진=유튜브 채널 '서울경찰' 영상 캡처
출근길 '묻지마 폭행'을 저지른 남성이 현행범으로 체포되는 모습이 공개됐다.
18일 서울경찰청 공식 유튜브 채널에 '출근길 묻지마 폭행범 검거 현장'이라는 제목으로 지난 3월 서울 관악구 골목길에서 난동을 부렸던 남성의 모습이 담긴 영상이 게재됐다.
이 남성은 상의를 벗어 문신이 그려진 몸으로 길을 걷던 한 여성을 향해 달려갔다. 갑자기 돌진하는 남성을 보며 놀란 여성이 횡단보도로 뛰어가며 도망쳤지만, 남성은 그 뒤를 쫓아갔다.
다른 행인들이 남성을 말렸음에도 난동은 이어졌다. 신고받고 현장에 출동한 경찰은 폭행 및 공무집행방해 등 혐의로 현장에서 체포했다.
/사진=유튜브 채널 '서울경찰' 영상 캡처
경찰의 등장에도 남성은 침을 뱉고, 욕을 하는 등 폭력적인 행동을 이어갔다. 결국 경찰은 남성을 힘으로 제압해 압송했다.
경찰에 따르면 이 남성과 여성은 일면식도 없는 사이였다. 경찰은 "마약사범으로 의심했으나 마약 성분은 검출되지 않았다"며 "관제센터와의 공조, 시민들의 도움으로 빠르게 찾아낸 덕에 2차 피해를 막을 수 있었다"고 전했다.
김소연 한경닷컴 기자 sue123@hankyung.com


텍스트에 '행동' 이 있다고, '전라남도 순천시 행동' 이 나온다...  
쉽지 않다.  

#### inspection index no.2 to fix

In [39]:
history[2].df_filtered

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN
28150,서울특별시,관악구,봉천동,NaN,NaN
28151,서울특별시,관악구,신림동,NaN,NaN
28152,서울특별시,관악구,남현동,NaN,NaN


In [38]:
history[2].df_level

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN


In [40]:
history[2].df_count

,index,lv1,lv2,lv3
0,11913,1,1,1


lv4 혹은 lv3 에서 확인된 주소일 때,  
그 윗 단계 위의 주소가 텍스트에 있는지 확인을 한다.  

#### check RA

In [1]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
from utils.search_address import RetrieveAddress, get_lat_long

In [2]:
df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
df_addr = pd.read_csv('./address/address.csv')

In [3]:
texts = df_hk_json.iloc[:]['text']

In [4]:
ra = RetrieveAddress(df_addr)

#### check RA - fix: 'index' key error

In [8]:
res = ra.process(item)

KeyError: 'index'

In [9]:
ra.df_filtered

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN
28150,서울특별시,관악구,봉천동,NaN,NaN
28151,서울특별시,관악구,신림동,NaN,NaN
28152,서울특별시,관악구,남현동,NaN,NaN


In [11]:
ra.df_level

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN


In [10]:
ra.df_count

,index,lv1,lv2,lv3
0,11913,1,1,1


In [12]:
idxmax = ra.df_count.iloc[ra.df_count[f'lv{ra.level - 1}'].idxmax()]        

In [13]:
idxmax

index    11913
lv1          1
lv2          1
lv3          1
Name: 0, dtype: int64

In [17]:
row = ra.df_address.iloc[idxmax['index']]
row

lv1    전라남도
lv2     순천시
lv3      행동
lv4     NaN
lv5     NaN
Name: 11913, dtype: object

#### check RA - modify 

In [5]:
res = ra.process(texts[2])

In [6]:
ra.df_filtered

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN
28150,서울특별시,관악구,봉천동,NaN,NaN
28151,서울특별시,관악구,신림동,NaN,NaN
28152,서울특별시,관악구,남현동,NaN,NaN


In [7]:
ra.df_level

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN


In [8]:
ra.df_count

,index,lv1,lv2,lv3
0,11913,1,1,1


#### ask: fruit

In [8]:
fruit = {
    'fruit_name': ['apple', 'banana', 'cherry'],
    'stock': [10, 5, 7]
}
df_fruit = pd.DataFrame(fruit)

In [9]:
df_fruit

,fruit_name,stock
0,apple,10
1,banana,5
2,cherry,7


In [15]:
text = '2 apples, 2 bananas, 1 more apple'

In [16]:
df_mask = df_fruit.map(
    lambda x: bool(
        re.search(
            r'\b{}'.format(str(x)),
            text,
            re.IGNORECASE)))

In [17]:
df_filtered = df_fruit[df_mask.any(axis=1)]

In [18]:
df_filtered

,fruit_name,stock
0,apple,10
1,banana,5


In [ ]:
fruit = {
    'fruit_name': ['apple', 'banana', 'cherry'],
    'stock': [10, 5, 7]
}
df_fruit = pd.DataFrame(fruit)

text = '2 apples, 2 bananas, 1 more apple'
df_mask = df_fruit.map(
    lambda x: bool(
        re.search(
            r'\b{}'.format(str(x)),
            text,
            re.IGNORECASE)))
df_filtered = df_fruit[df_mask.any(axis=1)]

In [23]:
# Initialize a dictionary to store keyword counts
keyword_counts = {keyword: 0 for keyword in df_fruit['fruit_name']}

# Iterate over DataFrame rows and count occurrences
for index, row in df_fruit.iterrows():
    keyword = row['fruit_name']
    pattern = r'\b{}'.format(re.escape(keyword))
    count = len(re.findall(pattern, text, re.IGNORECASE))
    keyword_counts[keyword] += count

print(keyword_counts)

{'apple': 2, 'banana': 1, 'cherry': 0}


In [24]:
# Convert the dictionary to a DataFrame
df_keyword_counts = pd.DataFrame.from_dict(keyword_counts, orient='index', columns=['count'])
df_keyword_counts

,count
apple,2
banana,1
cherry,0


#### check RA - apply:

In [25]:
texts[2]

'/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n출근길 \'묻지마 폭행\'을 저지른 남성이 현행범으로 체포되는 모습이 공개됐다.\n\n18일 서울경찰청 공식 유튜브 채널에 \'출근길 묻지마 폭행범 검거 현장\'이라는 제목으로 지난 3월 서울 관악구 골목길에서 난동을 부렸던 남성의 모습이 담긴 영상이 게재됐다.\n\n이 남성은 상의를 벗어 문신이 그려진 몸으로 길을 걷던 한 여성을 향해 달려갔다. 갑자기 돌진하는 남성을 보며 놀란 여성이 횡단보도로 뛰어가며 도망쳤지만, 남성은 그 뒤를 쫓아갔다.\n\n다른 행인들이 남성을 말렸음에도 난동은 이어졌다. 신고받고 현장에 출동한 경찰은 폭행 및 공무집행방해 등 혐의로 현장에서 체포했다.\n/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n경찰의 등장에도 남성은 침을 뱉고, 욕을 하는 등 폭력적인 행동을 이어갔다. 결국 경찰은 남성을 힘으로 제압해 압송했다.\n\n경찰에 따르면 이 남성과 여성은 일면식도 없는 사이였다. 경찰은 "마약사범으로 의심했으나 마약 성분은 검출되지 않았다"며 "관제센터와의 공조, 시민들의 도움으로 빠르게 찾아낸 덕에 2차 피해를 막을 수 있었다"고 전했다.\n\n김소연 한경닷컴 기자 sue123@hankyung.com'

In [27]:
mask = df_addr.map(
    lambda x: bool(
        re.search(
            r'\b{}'.format(str(x)),
            texts[2],
            re.IGNORECASE)))
df_filtered = df_addr[mask.any(axis=1)]
df_level = df_addr[mask[f'lv3']]

In [28]:
df_filtered

,lv1,lv2,lv3,lv4,lv5
11913,전라남도,순천시,행동,NaN,NaN
28150,서울특별시,관악구,봉천동,NaN,NaN
28151,서울특별시,관악구,신림동,NaN,NaN
28152,서울특별시,관악구,남현동,NaN,NaN


In [37]:
addr_name_set = set([])

for index, row in df_filtered.iterrows():
    for value in row.values:
        if isinstance(value, str):
            if value not in addr_name_set:
                addr_name_set.add(value)

In [38]:
addr_name_set

{'관악구', '남현동', '봉천동', '서울특별시', '순천시', '신림동', '전라남도', '행동'}

In [41]:
keyword_counts = {keyword: 0 for keyword in addr_name_set}

for keyword in addr_name_set:
    pattern = r'\b{}'.format(re.escape(keyword))
    count = len(re.findall(pattern, texts[2], re.IGNORECASE))
    keyword_counts[keyword] += count

print(keyword_counts)

{'순천시': 0, '행동': 1, '신림동': 0, '전라남도': 0, '서울특별시': 0, '남현동': 0, '봉천동': 0, '관악구': 1}


'서울특별시' 이거 먼저 좀 해결을 해야겠다.  

#### ask: replace contents in dataframe

In [42]:
food = {
    'item': ['apple', 'banana', 'cherry', 'onion', 'garlic', 'flour'],
    'stock': [10, 5, 7, 8, 12, 5]
}
df_food = pd.DataFrame(food)

# make 'code' column, 'fruit' for 'apple', 'banana', 'cherry' rows, 'vegetable' for 'onion', 'garlic', and 'grain' for 'flour'

In [43]:
# Define conditions and choices for the 'code' column
conditions = [
    df_food['item'].isin(['apple', 'banana', 'cherry']),
    df_food['item'].isin(['onion', 'garlic']),
    df_food['item'].isin(['flour'])
]
choices = ['fruit', 'vegetable', 'grain']

# Create the 'code' column based on conditions
df_food['code'] = np.select(conditions, choices, default='')

print(df_food)

     item  stock       code
0   apple     10      fruit
1  banana      5      fruit
2  cherry      7      fruit
3   onion      8  vegetable
4  garlic     12  vegetable
5   flour      5      grain


#### apply: df_address

In [44]:
df_addr.head(1)

,lv1,lv2,lv3,lv4,lv5
0,충청남도,천안시 동남구,대흥동,NaN,NaN


In [46]:
lv1_set = set([])

for index, row in df_addr.iterrows():
    value = row['lv1']
    if isinstance(value, str):
        if value not in lv1_set:
            lv1_set.add(value)

In [47]:
lv1_set

{'강원특별자치도',
 '경기도',
 '경상남도',
 '경상북도',
 '광주광역시',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '서울특별시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전북특별자치도',
 '제주특별자치도',
 '충청남도',
 '충청북도'}

In [50]:
lv0_set = [
    '강원', '경기', '경남', '경북', '광주',
    '대구', '대전', '부산', '서울', '울산',
    '인천', '전남', '전북', '제주', '충남',
    '충북'
]

In [49]:
conditions = [df_addr['lv1'].isin([item]) for item in lv1_set]

In [51]:
df_addr['lv0'] = np.select(conditions, lv0_set, default='')

In [52]:
df_addr.head()

,lv1,lv2,lv3,lv4,lv5,lv0
0,충청남도,천안시 동남구,대흥동,NaN,NaN,인천
1,충청남도,천안시 동남구,성황동,NaN,NaN,인천
2,충청남도,천안시 동남구,문화동,NaN,NaN,인천
3,충청남도,천안시 동남구,사직동,NaN,NaN,인천
4,충청남도,천안시 동남구,영성동,NaN,NaN,인천


뭐야 왜 '충청남도'를 '인천'으로 값을 매겼지?

In [53]:
df_addr.tail()

,lv1,lv2,lv3,lv4,lv5,lv0
28194,서울특별시,강동구,둔촌동,NaN,NaN,경북
28195,서울특별시,강동구,암사동,NaN,NaN,경북
28196,서울특별시,강동구,성내동,NaN,NaN,경북
28197,서울특별시,강동구,천호동,NaN,NaN,경북
28198,서울특별시,강동구,강일동,NaN,NaN,경북


In [54]:
for item in zip(lv1_set, lv0_set):
    ic(item)

ic| item: ('제주특별자치도', '강원')
ic| item: ('전라남도', '경기')
ic| item: ('광주광역시', '경남')
ic| item: ('서울특별시', '경북')
ic| item: ('대구광역시', '광주')
ic| item: ('강원특별자치도', '대구')
ic| item: ('울산광역시', '대전')
ic| item: ('경기도', '부산')
ic| item: ('경상남도', '서울')
ic| item: ('충청북도', '울산')
ic| item: ('충청남도', '인천')
ic| item: ('부산광역시', '전남')
ic| item: ('대전광역시', '전북')
ic| item: ('전북특별자치도', '제주')
ic| item: ('경상북도', '충남')
ic| item: ('인천광역시', '충북')


In [55]:
sorted(lv1_set)

['강원특별자치도',
 '경기도',
 '경상남도',
 '경상북도',
 '광주광역시',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '서울특별시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전북특별자치도',
 '제주특별자치도',
 '충청남도',
 '충청북도']

In [56]:
conditions = [df_addr['lv1'].isin([item]) for item in sorted(lv1_set)]

In [57]:
df_addr['lv0'] = np.select(conditions, lv0_set, default='')

In [58]:
df_addr.head()

,lv1,lv2,lv3,lv4,lv5,lv0
0,충청남도,천안시 동남구,대흥동,NaN,NaN,충남
1,충청남도,천안시 동남구,성황동,NaN,NaN,충남
2,충청남도,천안시 동남구,문화동,NaN,NaN,충남
3,충청남도,천안시 동남구,사직동,NaN,NaN,충남
4,충청남도,천안시 동남구,영성동,NaN,NaN,충남


In [59]:
df_addr.tail()

,lv1,lv2,lv3,lv4,lv5,lv0
28194,서울특별시,강동구,둔촌동,NaN,NaN,서울
28195,서울특별시,강동구,암사동,NaN,NaN,서울
28196,서울특별시,강동구,성내동,NaN,NaN,서울
28197,서울특별시,강동구,천호동,NaN,NaN,서울
28198,서울특별시,강동구,강일동,NaN,NaN,서울


In [60]:
# Extract the last column
last_column = df_addr.pop('lv0')

# Insert the last column at the beginning
df_addr.insert(0, 'lv0', last_column)

df_addr.head()

,lv0,lv1,lv2,lv3,lv4,lv5
0,충남,충청남도,천안시 동남구,대흥동,NaN,NaN
1,충남,충청남도,천안시 동남구,성황동,NaN,NaN
2,충남,충청남도,천안시 동남구,문화동,NaN,NaN
3,충남,충청남도,천안시 동남구,사직동,NaN,NaN
4,충남,충청남도,천안시 동남구,영성동,NaN,NaN


#### save to parquet

In [61]:
df_addr.to_parquet('./address/df_addr.parquet.gzip', compression='gzip')

In [62]:
df_addr_parquet = pd.read_parquet('./address/df_addr.parquet.gzip')

In [63]:
df_addr_parquet.head()

,lv0,lv1,lv2,lv3,lv4,lv5
0,충남,충청남도,천안시 동남구,대흥동,None,NaN
1,충남,충청남도,천안시 동남구,성황동,None,NaN
2,충남,충청남도,천안시 동남구,문화동,None,NaN
3,충남,충청남도,천안시 동남구,사직동,None,NaN
4,충남,충청남도,천안시 동남구,영성동,None,NaN


#### continue: check RA

In [64]:
mask = df_addr.map(
    lambda x: bool(
        re.search(
            r'\b{}'.format(str(x)),
            texts[2],
            re.IGNORECASE)))
df_filtered = df_addr[mask.any(axis=1)]
df_level = df_addr[mask[f'lv3']]

In [67]:
addr_name_set = set([])

for index, row in df_filtered.iterrows():
    for value in row.values:
        if isinstance(value, str):
            if value not in addr_name_set:
                addr_name_set.add(value)

In [ ]:
keyword_counts = {keyword: 0 for keyword in addr_name_set}

for keyword in addr_name_set:
    pattern = r'\b{}'.format(re.escape(keyword))
    count = len(re.findall(pattern, texts[2], re.IGNORECASE))
    keyword_counts[keyword] += count

In [69]:
print(keyword_counts)

{'아현동': 0, '죽림동': 0, '남영동': 0, '망원동': 0, '침산동': 0, '남산동3가': 0, '보수동1가': 0, '길동': 0, '남포동5가': 0, '방학동': 0, '의주로2가': 0, '상덕동': 0, '청진동': 0, '봉래동2가': 0, '이촌동': 0, '동광동1가': 0, '청담동': 0, '부사동': 0, '삼선동2가': 0, '마포동': 0, '인사동': 0, '운북동': 0, '공평동': 0, '남산동': 0, '금천구': 0, '광복동2가': 0, '태평로3가': 0, '목달동': 0, '고척동': 0, '성북구': 0, '중학동': 0, '산림동': 0, '동인동4가': 0, '향촌동': 0, '용강동': 0, '용답동': 0, '보광동': 0, '덕교동': 0, '동숭동': 0, '동소문동1가': 0, '신대방동': 0, '반포동': 0, '보문동7가': 0, '하월곡동': 0, '용문동': 0, '항동': 0, '종로3가': 0, '주교동': 0, '양평동': 0, '고덕동': 0, '성수동1가': 0, '용동': 0, '성남동': 0, '안암동3가': 0, '본동': 0, '홍익동': 0, '초동': 0, '무학동': 0, '금호동1가': 0, '남포동6가': 0, '가양동': 0, '학성동': 0, '염리동': 0, '관철동': 0, '안암동5가': 0, '당산동4가': 0, '당산동5가': 0, '남창동': 0, '이태원동': 0, '식만동': 0, '신영동': 0, '계동': 0, '대안동': 0, '독산동': 0, '신흥동2가': 0, '양평동2가': 0, '사정동': 0, '학산동': 0, '양재동': 0, '문배동': 0, '보문동1가': 0, '봉림동': 0, '토정동': 0, '풍납동': 0, '광장동': 0, '연남동': 0, '장관동': 0, '저동2가': 0, '묵정동': 0, '합정동': 0, '장교동': 0, '을지로6가': 0, '양평동4가': 0, '남북동': 0, '사간동': 0, '

In [66]:
df_filtered

,lv0,lv1,lv2,lv3,lv4,lv5
11913,전남,전라남도,순천시,행동,NaN,NaN
27516,서울,서울특별시,종로구,청운동,NaN,NaN
27517,서울,서울특별시,종로구,신교동,NaN,NaN
27518,서울,서울특별시,종로구,궁정동,NaN,NaN
27519,서울,서울특별시,종로구,효자동,NaN,NaN
...,...,...,...,...,...,...
28194,서울,서울특별시,강동구,둔촌동,NaN,NaN
28195,서울,서울특별시,강동구,암사동,NaN,NaN
28196,서울,서울특별시,강동구,성내동,NaN,NaN
28197,서울,서울특별시,강동구,천호동,NaN,NaN


In [71]:
df_keyword_counts = pd.DataFrame.from_dict(keyword_counts, orient='index', columns=['count'])

In [72]:
df_keyword_counts[df_keyword_counts['count'] > 0]

,count
관악구,1
서울,4
행동,1
